In [ ]:
import os
import azureml
from shutil import copyfile, rmtree
from pathlib import Path
import urllib.request

from azureml.train.estimator import Estimator
from azureml.core import Workspace, Datastore, Experiment, Environment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.resource_configuration import ResourceConfiguration

import spacy
from nlp_architect.models.absa.inference.inference import SentimentInference
from spacy import displacy
from nlp_architect.models.absa.inference.data_types import TermType

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
cluster_name     = "absa-cluster"
experiment_name  = "absa"
model_name       = "absa"
aci_deploy_name  = "absa-aci"

#   
#   
#   
# Step 1 - Setup your environment
#   
![title](https://raw.githubusercontent.com/microsoft/ignite-learning-paths-training-aiml/master/aiml40/absa/notebooks/images/step1.png)

## Initialize workspace

In [ ]:
try:
    ws = Workspace.from_config()
    print("Using workspace:",ws.name,"in region", ws.location)
except:
    print('Workspace not found')

## Attach a Compute Cluster

In [ ]:
# Choose a name for your CPU cluster
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Using compute cluster:', cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           vm_priority='lowpriority',
                                                           min_nodes=0,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion(show_output=True)

## Attach a Datastore

In [ ]:
# Connect to the default datastore
ds = ws.get_default_datastore()

#   
#   
# Step 2 - Experiment with your data & models
#   
![title](https://raw.githubusercontent.com/microsoft/ignite-learning-paths-training-aiml/master/aiml40/absa/notebooks/images/step2.png)

## Create an Experiment

In [ ]:
# Create an experiment or connect if it exists
exp = Experiment(workspace=ws, name=experiment_name)

## Create and run an Estimator

In [ ]:
script_params = {
    '--data_folder': ds,
    '--large': 'yes'
}

nlp_est = Estimator(source_directory='../scripts',
                   script_params=script_params,
                   compute_target=cluster,
                   environment_variables = {'NLP_ARCHITECT_BE':'CPU'},
                   entry_script='train.py',
                   pip_packages=['git+https://github.com/NervanaSystems/nlp-architect.git@absa',
                                 'spacy==2.1.8']
)

In [ ]:
# Submit and run the estimator
run = exp.submit(nlp_est)

In [ ]:
RunDetails(run).show()

In [ ]:
# Select a previous run and show the details
run = [r for r in exp.get_runs() if r.id == 'absa_1580996552_8fcdf147'][0]
RunDetails(run).show()

### Register Model

In [ ]:
model = run.register_model(model_name = model_name, 
                     model_path = 'outputs', 
                     model_framework =  Model.Framework.TENSORFLOW,
                     model_framework_version = '1.13',
                     description = 'Aspect Based Sentiment Analysis - Intel',
                     tags={'area': 'NLP', 'type': 'unsupervised', 'model_author': "INTEL"},
                     resource_configuration = ResourceConfiguration(cpu=1, memory_in_gb=2))

print('version:',model.version)

### Test our model On Sample Data 

In [ ]:
# Download the model 
# model = Model(ws, 'absa')
Path("../temp").mkdir(parents=True, exist_ok=True)
model.download(exist_ok=True,target_dir="../temp")

In [ ]:
# load the model
# When you run for the first time, you will need to run twice (Spacy install)
c_aspect_lex = '../temp/outputs/generated_aspect_lex.csv'
c_opinion_lex = '../temp/outputs/generated_opinion_lex_reranked.csv' 
inference = SentimentInference(c_aspect_lex, c_opinion_lex)

In [ ]:
# Run the predictions
docs = ["Loved the sweater but hated the pants",
       "Really great outfit, I really like the shirt",
       "The shoes are bad, but perfect blouse"]

sentiment_docs = []

for doc_raw in docs:
    sentiment_doc = inference.run(doc=doc_raw)
    sentiment_docs.append(sentiment_doc)

In [ ]:
# Visualize the predictions on the data
ents = []
for doc in sentiment_docs:    
    doc_viz = {'text':doc._doc_text, 'ents':[]}
    for s in doc._sentences:
        for ev in s._events:
            for e in ev:
                if e._type == TermType.ASPECT:
                    ent = {'start': e._start, 'end': e._start + e._len,'label':str(e._polarity.value), 'text':str(e._text)}
                    if all(kown_e['start'] != ent['start'] for kown_e in ents):
                        ents.append(ent)
                        doc_viz['ents'].append(ent)
    doc_viz['ents'].sort(key=lambda m: m["start"])
    displacy.render(doc_viz, style="ent", options={'colors':{'POS':'#7CFC00', 'NEG':'#FF0000'}}, manual=True, jupyter=True)

#   
#   
#   
# Step 3 -  Deploy your model in Azure

In [ ]:
# Load the latest registered model
model = Model(ws, name=model_name)
print("Loaded model version:",model.version)

### Create an environment file

In [ ]:
pip = ["azureml-defaults", 
       "azureml-monitoring", 
       "git+https://github.com/NervanaSystems/nlp-architect.git@absa", 
       "spacy==2.1.8"]

myenv = CondaDependencies.create(pip_packages=pip)

Path("../temp").mkdir(parents=True, exist_ok=True)

with open("../temp/myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
deploy_env = Environment.from_conda_specification('absa_env', "../temp/myenv.yml")
deploy_env.environment_variables={'NLP_ARCHITECT_BE': 'CPU'}

### Create an Inference config

In [ ]:
inference_config = InferenceConfig(environment=deploy_env, entry_script="../scripts/score.py")

### Deploy to ACI

In [ ]:
# Create a deployment config
deploy_config = AciWebservice.deploy_configuration(
                    cpu_cores = model.resource_configuration.cpu, 
                    memory_gb = model.resource_configuration.memory_in_gb,
                    description='Aspect-Based Sentiment Analysis - Intel')

In [ ]:
# Deploy the model to an ACI
deployment = Model.deploy(ws, 
                name=aci_deploy_name, 
                models = [model], 
                inference_config = inference_config, 
                deployment_config = deploy_config, 
                overwrite = True)

deployment.wait_for_deployment(show_output=True)

### Get the scoring details

In [ ]:
# Connect to previous deployment
deployment = AciWebservice(ws, aci_deploy_name)

In [ ]:
print("Scoring endpoint:",deployment.scoring_uri)
print("Test uri:","http://aiml40.azurewebsites.net/?url="+deployment.scoring_uri)